# Home assignment 2

You should work on the assignement in groups/teams of 3 participants. Submissions of single students will not be accepted! Please use the Forum in case of doubt in order to find team mates!

Upload your solution as a jupyter notebook to moodle by Tuesday, 7th of January 23:55h. (The deadline is strict)
It is sufficient if one student of each team submits the solution.


You should add comments to your code where necessary and print the relevant results. You should also always test your code on self-chosen examples.

Do not forget to specify the (First_name, Last_name, student_id (matrikelnummer)) of all contributing students in the jupyter notebook here:

Student 1: `First_name, Last_name, student_id`

Student 2: `Qingqing, Yang, 393415`

Student 3: `First_name, Last_name, student_id`

# Implementing GloVe
In this task you will implement the glove algorithm using PyTorch. (One advantage is that you need not calculate gradient by hand, but you can take advantage of the autograd module). The task will require implementation of certain functions, which we look into step-by-step:

Define a function for word to index mapping. Since the model won't be able to take strings as input we will convert them into indices. The function will generate a mapping  w2i which uses words as keys and corresponding indices as values e.g., `w2i['walk'] = 42`. As Preprocessing, remove all punctuations and convert all words to lower case.

In [1]:
import string
from nltk.tokenize import word_tokenize

def word2indexMapping(textfile):
    w2i = {}
    text = [] # sequence of words as they appear in the text after removing punctuations
    
    #Preprocessing
    textfile = textfile.replace('\n', ' ').strip()
    translator = str.maketrans(dict.fromkeys(string.punctuation))
    text = textfile.translate(translator)
    text = text.lower()
    
    #Mapping
    test_list = word_tokenize(text)
    w2i = {k: test_list.index(k) for k in test_list}

    return w2i, text


Define a function for calculating a two dimensional matrix $X_{ij}$ which is the number of times word $j$ occurred in the context of word $i$. The size of the context window $k$ (as a number of words, $k=2$ describes that the context contains the two words before and the two words after the central word) is also an argument of the function. 


In [2]:
import numpy as np

In [3]:
def co_occurrenceFreq(text, w2i, k): # text is a sequence of words ordered as they appear in the text
    # note that there is no notion of sentence here...
    
    X_ij =  np.zeros((len(w2i), len(w2i)))
    
    # write your code snippet here...
    text = text.split(" ")
    tokens = list(w2i.keys())
    token_ids = list(w2i.values())
    context_ids = [] #context for each word
    for center_id in token_ids:
        #left part context for given center
        x = max(0, center_id - k)
        #right part context for given center_id
        y = min(len(text) - 1, center_id + k)
        context_ids.append(list(range(x, y + 1)))
        contexts_len = len(context_ids)
    #print(context_ids)
    count, y = 0, 0 #y represent for index for each center word
    for center_i in tokens:
        for i in range(0,contexts_len): #two demension list interation by i, j
            count = 0
            for j in range(0,len(context_ids[i])):
                if center_i!=text[context_ids[i][j]]:
                    j=j+1
                else: count=count+1
            X_ij[i][y]=count
            i+=1
        y+=1
    return X_ij

Define a GloVe model class with parameters $w$, $\hat w$, $b$ and $\hat b$. For a particular pair of words $i$, $j$, your forward function should implement $w_{i}^{T}\hat w_{j} + b_{i} + \hat b_{j}$. Assume a dimension of embedding to be $d$ which you will specify when creating an instance of the model.

In [4]:
import torch.nn as nn
import torch

In [5]:
class Glove(nn.Module):
    # write your model class here....
    def __init__(self, text, dimension):
        super().__init__()
        self.dimension = dimension
        self.vocab_size = len(text) #text after preprocess, so only unique words contained
        #Word vector matrix (2V) * d
        #All elements are initialized randomly in the range (-0.5,0.5]
        self.w_1 = nn.Parameter((torch.randn(self.vocab_size * 2, dimension) - 0.5) / float(dimension + 1), requires_grad=True)
        self.w_2 = nn.Parameter((torch.randn(self.vocab_size * 2, dimension) - 0.5) / float(dimension + 1), requires_grad=True)
        #bias array size (2V)
        #initialized randomly in the range (-0.5, 0.5]
        self.b_1 = nn.Parameter((torch.randn(self.vocab_size * 2) - 0.5) / float(dimension + 1), requires_grad=True)
        self.b_2 = nn.Parameter((torch.randn(self.vocab_size * 2) - 0.5) / float(dimension + 1), requires_grad=True)

        
    def forward(self, i, j):
        cost = self.w_1[i].dot(self.w_2[j]) + self.b_1[i] + self.b_2[j]#first transpose, then multiply, add
        return cost

    pass

Write a function that implements the weighting function $f(X_{ij})$

$f(x) = (\frac{x}{100})^{\frac{3}{4}}$ if x<100 

$f(x) = 1 $ otherwise

In [6]:
def weightFunction(X_ij, i, j):
    f = 0
    # write your code snippet here
    x_max = 100
    alpha = 0.75
    f = (X_ij[i][j] / x_max) ** alpha if X_ij[i][j] < x_max else 1
    return f

Write a train function to train the model using stochastic gradient descent. Your each training instance would be a word-context pair ($i$, $j$) and the corresponding loss function would be 

$f(X_{ij})(w_{i}^{T}\hat w_{j} + b_{i} + \hat b_{j} - log(1 + X_{ij}))^2$

In [7]:
from torch import optim
import math

def loss_func(X_ij, i, j):
    return weightFunction(X_ij, i, j)*((model.forward(i, j)-math.log(1+X_ij[i][j]))**2)

def train(model, X_ij, learning_rate=0.001, epochs=5):
    opt = optim.Adam(model.parameters(), lr=0.001) # use Adam as your optimizer
    for _ in range(epochs):
        # train across each word-conext pair
        for i in range(len(X_ij)): 
            for j in range(len(X_ij[0])):
                loss = loss_func(X_ij, i, j)# calculate loss
                loss.backward() # backpropagate for every training example
                opt.step()
                opt.zero_grad()
                #print(loss_func(X_ij, i, j))
    return model.w_1, model.w_2, model.b_1, model.b_2        

Write a function to generate embeddings given a word. The embedding of a word with index $i$ would $w_i + \hat w_i$

In [8]:
def getEmbedding(model, word):
    # write your code snippet here...
    index = w2i[word]
    embed = model.w_1[index] + model.w_2[index]
    return embed

Put the individual components together to train a Glove model from the given text file  'shakespeare-caesar.txt' with dimension 200 and a context window of 5.
Manually inspect nearest neigbors for some self-picked words.

In [11]:
with open('/Users/yangqingqing/Downloads/data_home_assignment_2/shakespeare-caesar.txt', encoding='latin-1') as f:
    Text = f.read()
w2i, text = word2indexMapping(Text)
X_ij = co_occurrenceFreq(text, w2i, k = 5)
model = Glove(text, dimension=10)
print(model.w_1, model.w_2, model.b_1, model.b_2)
train(model, X_ij, learning_rate=0.001, epochs=5)
print(model.w_1, model.w_2, model.b_1, model.b_2)

Parameter containing:
tensor([[-0.0657, -0.0572, -0.1281,  ..., -0.0904, -0.1343, -0.0241],
        [-0.1670,  0.0372, -0.1388,  ..., -0.1712, -0.1954, -0.2449],
        [-0.1265, -0.0847, -0.0052,  ...,  0.0859, -0.2149, -0.0122],
        ...,
        [-0.0480, -0.1029,  0.0109,  ..., -0.0419, -0.0789, -0.2526],
        [-0.1959, -0.1430, -0.0826,  ..., -0.0165, -0.0856,  0.1222],
        [-0.0970,  0.2424, -0.0202,  ...,  0.1278, -0.0244, -0.1326]],
       requires_grad=True)


KeyboardInterrupt: 

# Word embeddings with gensim

Gensim is a package which allows you to train word embeddings as well. The task is to take a text file (like 'shakespeare-caesar.txt') and generate embeddings of the vocabulary. You can consult the documentation here - 
https://radimrehurek.com/gensim/models/word2vec.html

# Word similarity with pre-trained embeddings

One way of understanding how good the obtained embeddings are is the word similarity task. The file 'sim353.csv' contains a set of word pairs as well as their similarity as judged by humans (e.g., tiger,tiger,10). 
Also we provide a set of pre-trained embeddings in 'embeddings.pickle' in the form of a dictionary with words as keys and embeddings as values. 

Step 1: Consider each word pair in the given file ('sim353.csv') and calculate the cosine similarity between them and then rank the word pairs based on the similarity value

Step 2: Rank the word pairs based on the similarity values as determined by humans

Step 3: Calculate the spearman rank correlation between the two ranked list of word-pairs.

# Text classification with CNNs

You are given a set of news articles which are to be labelled 1 or 0. The data is split into 3 groups (train/test/validation). Each group is further divided into 2 files which consists of the text(ending with \_X.p) and the label (ending with \_y.p) respectively. Each datapoint is a list of words and they are all accumulated in a list forming a list of lists. The label file is a list of labels (0/1). 

We will design a character-level CNN. So the first task would be to obtain a one-hot encoding for the characters. 

In [ ]:
# the vocabulary of caracters is provided for your reference.. 
vocabulary = list(""" abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:'/\|_@#$%ˆ&*˜‘+-=<>()[]{}""")

In [ ]:
def characterEncoding(vocabulary):
    c2v = {} # dictionary with key as a character and one-hot encoding as value
    # write your code snippet here..
    return c2v

Encode a sentence as a 2D matrix with each row representing the one-hot encoding of a character in the sequence.

In [ ]:
def sentence2tensor(sentence,c2v):
    # write your code snippet here

Define a dataset class for feeding data to your model

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class NewsDataset(Dataset):
    # write your code snippet here...

Define a model class with 2 layers of convolutions each followed by a ReLU unit. The model should have linear layer which maps to the classes. 

In [ ]:
class Classifier(nn.Module):
    # write your code snippet here

Define a train function which trains on the train dataset. You can use binary cross entropy as your loss function and Adam as your optimizer.

In [ ]:
def train(model, train_dataset, epochs=5, batchsize=32, learning_rate=0.0001):
    # write your code snippet here

Define a test function which takes the trained model and test dataset and outputs the accuracy.

In [ ]:
def test(model, test_datset, batchsize=32):
    # write your code snippet here

In [ ]:
# putting it altogether

train_dataset = 
test_dataset = 

model = Classifier()

train(model, train_datset)
test(model, test_dataset)